## Méthodo


Pour gagner du temps, penser à: 

Faire fonctionner fonctins de tests en bas des fichiers `.py`

Exemple: après une modification de l'agent, faire tourner `test_losses()` et vérifier que la loss baisse toujours. 


Dur équilibre entre 
* "garder les options passées"
* "garder un code pas trop gros" (en supprimant les fonctions devenues inutile). 
* Limiter la duplication de code. Il y avait d'ailleurs 2 chemins fonctionnels pour faire tourner le solver (un pour le test, un pour le train). Très embêtant.  


## Accélération tensorflow

Transformation des programmes pour que le `@tf.function` fonctionne:

* Le calcul des gradients
* La projection de toutes les solution : Attention, si l'on veut passer des jeux de données plus gros il faudrait quand même faire un découpage sur l'indice de batch (pas trop dur). 

* La prédiction (qui donne aussi le score)

* le solver

* Le réseau de neurone=> changement de structure informatique


Non accélérer: la génération des données initiales




## Génération des fonctions initiales: 


La génération de données se fait à l'aide de simple fonctions

### `init_random_sod`

Les random-sod sont beaucoup plus méchants, pour imiter la difficulté des sods.  

        def init_random_sod(param:Param, batch_size:int, k:K, minimum_E=0.05, maximal_jump_E=2, minimum_rho=0.05, maximal_jump_rho=2):


###  `init_periodic` (anciennement `loop`). 

### `init_non_periodic` (anciennement `changing)`. 


Les sauts de pression et de densité ont lieux au même moment, sont dans le même sens et du même ordre de grandeurs, mais ne sont pas  identique (comme c'était le cas dans le code de Victor). 

TODO: modifier un peu la génération de la partie continue du `non_periodic`, car elle est bêtement périodique. 


### Dans le notebook

Les différents types de fonctions peuvent être mélanger dans le notebook.

Dans le programme précédent: elles l'étaient dans l'objet de création des fonction d'une manière un peu  opaque.

Dans le programme précédent: Je n'ai pas vu où étaient mis les random_sod dans l'apprentissage ?






## Conditions aux bord


J'ai enfin compris ce qui aller pas avec `BC_reflexive`: Elle peut donner des densité et pressions négatives !  J'ai composée la réflexion sur les bords avec un relu lissé. On a alors des conditions au bords très régulière.  


Attention: si on utilise `BC_periodic` sur des conditions initiales non périodique, c'est pas du tout naturelle à mon avis. On a des jeux de données non génératique qui ont tous une discontinuité "non-controlée" sur les bords. 


La `BC_reflexive` peut aussi s'utiliser sur les sod: les solutions sembles plus naturelles quand le choc arrive sur les bords. 


Mais le choix de la solution au bord est avant tout un choix de modélisation !  donc il faut que l'on se demande ce que l'on veut modéliser...


## Solver



Il faudrait ré-intégrer les ordres >1. Avec l'accélération tensorflow, ça devrait allé vite.


Attention: respecter les restrictions liée à l'accélération: 
* Pas d'argument globaux à par self.model
* Les bornes des grandes boucles en tenseur
* Utiliser les `tf.TensorArray` pour les liste 
En gros: faides des fonctions sur le même modèle que les 

Il faudrait aussi l'intégrer dans l'agent, dans la méthode `gradient_tape`




## Réseau de neurone


Pour permettre l'accélération, la structure informatique du réseau à été complètement changé. 



3 modèles possibles: 

1. Difference_model_tricky: c'est exactement l'équivalent de l'ancien. Avant : fenêtre glissante en numpy. Puis chaque fenêtre était passé dans le réseau de neurone. Maintenant un `tricky_kernel` à base de matrices identité concatétée effectue la fenêtre glissante à l'intérieur même du réseau de neurone. ça va plus vite. 
2. `Difference_model_convo` : 1 couche de convolution crée la fenètre glissante, mais en même temps elle effectue une multiplication matricielle. C'est vraiment idem que le précédent: encore plus rapide. C'est just en peu moins interprétable si l'on cherche à faire des stats sur les différentes entrées. 

3.  `nn.Difference_model_several_convo`. Plusieurs couches de convolutions. Le plus prometteur à mon avis, mais l'on perd le côté local. Chaque prédiction utilise une fenêtre de taille: 


    (kernel_size-1)*len(conv_struct) + 1

Par défaut `kernel_size=4`, `conv_struct=(64,32,32)`. 







## Modif sur l'agent

* J'ai mis des scores $L^1, L^2 , L^{\infty}$. 


Les score $L^{\infty}$ sont parfois mauvais:  TODO: tester une distance  $L^{\infty} + L^2$ pour la loss. On pourrait même mettre des coeffients-famparams devant les 2 termes, coef modifié par popup. Mais il faut se fixer un score imuable quand même. 


* La distance entre solutions pour la validation est calculée tout le long du temps: car avec le temps certaines solutions s'applatissent, et l'erreur baisse naturellement. 

## Modif sur popup

Changement de la définition des décadents.

score_courant: moyenne des 3 dernier scores. 

Déclarer décadent un agent tel que le score_courant est dans les `weak` et le score max dans les `strong`

Quand un agent est décadent on le fait repartir de son top niveau. `on_decadence_perturb_me()`

Au bout de 2 ou 3 décadence consécuvite, il dégage. 

Quand tu appelles la méthode `get_best_agent(including_decadent=True)`





